## Ur10

In [2]:
import numpy as np
import pinocchio as pin
import casadi
from pinocchio import casadi as cpin
import example_robot_data as robex

In [151]:
# %load tp3/generated/free_ur10
robot = robex.load("ur10")

# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()

In [152]:
# create casadi model
cmodel = cpin.Model(model)
cdata = cmodel.createData()

nq = model.nq
nv = model.nv
nx = nq + nv
ndx = 2 * nv
cx = casadi.SX.sym("x", nx, 1)
cdx = casadi.SX.sym("dx", nv * 2, 1)
cq = cx[:nq]
cv = cx[nq:]
caq = casadi.SX.sym("a", nv, 1)
ctauq = casadi.SX.sym("tau", nv, 1)

# Compute kinematics casadi graphs
cpin.aba(cmodel, cdata, cq, cv, ctauq)
cpin.forwardKinematics(cmodel, cdata, cq, cv, caq)
cpin.updateFramePlacements(cmodel, cdata)
caba = casadi.Function("aba", [cx, ctauq], [cdata.ddq])

    

In [153]:
# compare pinocchio and casadi pinoccio
model.lowerPositionLimit = -3.14/2* np.ones((model.nq,1))
model.upperPositionLimit = 3.14/2*np.ones((model.nq,1))

v = np.zeros((model.nv,1))
q = np.zeros((model.nq,1))
print(q.shape)
print(v.shape)
tau = np.array([1.89358173e-08, -1.20801347e+02, -3.40055820e+01,  3.61284849e-08, -4.61613871e-12,  1.41448533e-09])

# Evaluate the derivatives
pin.computeABADerivatives(model,data,q,v,tau)
print(data.ddq)

caba = casadi.Function("aba", [cx, ctauq], [cdata.ddq])
print(caba([0,0,0,0,0,0,0,0,0,0,0,0], tau))


(6, 1)
(6, 1)
[ 7.59145875e-10  2.07937065e-06  4.76051472e-07  1.31250249e-07
 -9.42681928e-15  1.01889773e-10]
[7.59146e-10, 2.07937e-06, 4.76051e-07, 1.3125e-07, -9.42682e-15, 1.0189e-10]


In [259]:
target_acc = np.array([0.0,0.0,0.0,0.0,0.0,0.0])

# q, dq, can be modified for different initial value
x0 = np.array([0,0,0,0,0,0,0,0,0,0,0,0]) 
error_fcn = casadi.Function(
    "error_fcn",
    [ctauq],
    [casadi.sumsqr(target_acc - caba(x0,ctauq))],
)

opti = casadi.Opti()
var_torque = opti.variable(6,1)

opti.minimize(error_fcn(var_torque))
opti.subject_to(opti.bounded(-150, var_torque, 150))
opti.set_initial(var_torque, 20)
 
opti.solver('ipopt')
 
sol = opti.solve()
 
torque_result = sol.value(var_torque)
print(torque_result)
print(caba([0,0,0,0,0,0,0,0,0,0,0,0], torque_result))

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.4541186e+09 0.00e+00 5.00e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## YU

In [3]:
name = 'yu'
# load urdf file
urdf_file = '/home/anran.zhang/code/test_robot_models/yu.urdf'
 
# build model and data
model = pin.buildModelFromUrdf(urdf_file)
data = model.createData()
print('model name:' + model.name )

cmodel = cpin.Model(model)
cdata = cmodel.createData()

cq = casadi.SX.sym("q", model.nq)
cq_dot = casadi.SX.sym("q_dot", model.nq)
cx = casadi.vertcat(cq, cq_dot) # states

ctauq= casadi.SX.sym("tau", model.nq) # toqures(input)

q_dot_ = casadi.SX.sym("q_dot", model.nq)
q_ddot_ = casadi.SX.sym("q_ddot", model.nq)
x_dot = casadi.vertcat(q_dot_, q_ddot_)

caq = casadi.SX.sym("a", model.nq, 1)

# Compute kinematics casadi graphs
cpin.aba(cmodel, cdata, cq, cq_dot, ctauq)
cpin.forwardKinematics(cmodel, cdata, cq, cq_dot, caq)
cpin.updateFramePlacements(cmodel, cdata)

caba = casadi.Function("aba", [cx, ctauq], [cdata.ddq])



model name:yu


In [4]:
# compare direct pinocchio and cpinocchio(actual results used)

# using raw pinocchio
v = np.zeros((model.nv,1))
q = np.zeros((model.nq,1))
print(q.shape)
print(v.shape)
tau = np.array([-1.88529766e-07, -6.75996312e+01, -2.42971154e+01,  1.07061840e+00,-4.15493227e-08, 7.36708332e-01])
# Evaluate the derivatives
pin.computeABADerivatives(model,data,q,v,tau)

# using cpin
print(data.ddq)
print(caba([0,0,0,0,0,0,0,0,0,0,0,0],tau))


(6, 1)
(6, 1)
[-1.51305557e-08  4.27991338e-07  2.02169637e-07 -4.34805525e-09
  6.91497837e-09  1.25539074e-07]
[-1.51306e-08, 4.27991e-07, 2.0217e-07, -4.34792e-09, 6.91506e-09, 1.25539e-07]


In [17]:
target_acc = np.array([0.0,0.0,0.0,0.0,0.0,0.0])

# q, dq, can be modified for different initial value
x0 = np.array([0,0,0,0,0,0,0,0,0,0,0,0]) +0.0001*np.random.random((12,1))
error_fcn_yu = casadi.Function(
    "error_fcn_yu",
    [ctauq],
    [casadi.sumsqr(target_acc - caba(x0,ctauq))],
)

opti = casadi.Opti()
var_torque = opti.variable(model.nq,1)

opti.minimize(error_fcn_yu(var_torque))
opti.subject_to(opti.bounded(-150, var_torque, 150))
opti.set_initial(var_torque, 0)
 
opti.solver('ipopt')
 
sol = opti.solve()
 
torque_result = sol.value(var_torque)
print(torque_result)
print(caba([0,0,0,0,0,0,0,0,0,0,0,0], torque_result))

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0996163e+04 0.00e+00 5.00e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## Panda

In [19]:
# %load tp3/generated/free_ur10
robot = robex.load("panda")

# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()

In [20]:
# %load tp3/generated/free_helpers
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

nq = model.nq
nv = model.nv
nx = nq + nv
ndx = nx


cx = casadi.SX.sym("x", nx, 1)
cdx = casadi.SX.sym("dx", nx, 1)
cq = cx[:nq]
cv = cx[nq:]
caq = casadi.SX.sym("a", nv, 1)
ctauq = casadi.SX.sym("tau", nv, 1)

# Compute kinematics casadi graphs
cpin.aba(cmodel, cdata, cq, cv, ctauq)
cpin.forwardKinematics(cmodel, cdata, cq, cv, caq)
cpin.updateFramePlacements(cmodel, cdata)

caba = casadi.Function("aba", [cx, ctauq], [cdata.ddq])


In [21]:
target_acc = np.array([0.0]*model.nq)

# q, dq, can be modified for different initial value
# x0 = np.array([0.0] * nx)
x0 = [0.00000e+00, -0.78,  0.00000e+00, -2.3,  0.00000e+00, 1.57070, 0.78,  0.00000e-03,  0.00000e+00,
      0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

error_fcn_panda = casadi.Function(
    "error_fcn_panda",
    [ctauq],
    [casadi.sumsqr(target_acc - caba(x0,ctauq))],
)

opti = casadi.Opti()
var_torque = opti.variable(nv,1)

opti.minimize(error_fcn_panda(var_torque))
opti.subject_to(opti.bounded(-150, var_torque, 150))
opti.set_initial(var_torque, 100)
 
opti.solver('ipopt')
 
sol = opti.solve()
 
torque_result = sol.value(var_torque)
print(torque_result)
print('results acceleration')
print(error_fcn_panda(torque_result))
# print(error_fcn_panda([-3.49740277,-4.03988651e+00, -3.42179339e-09, -3.26685612e+00, -2.79824263e-09,  2.29967156e+00,  1.92950351e-11,  3.27483146e-10,-3.27483146e-10]))

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        9
Number of nonzeros in Lagrangian Hessian.............:       45

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        9
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        9
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.5091817e+08 0.00e+00 5.00e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## Kinova

In [3]:
# %load tp3/generated/free_ur10
robot = robex.load("kinova")

# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()

In [4]:
# %load tp3/generated/free_helpers
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

nq = model.nq
nv = model.nv
nx = nq + nv
ndx = nx


cx = casadi.SX.sym("x", nx, 1)
cdx = casadi.SX.sym("dx", nx, 1)
cq = cx[:nq]
cv = cx[nq:]
caq = casadi.SX.sym("a", nv, 1)
ctauq = casadi.SX.sym("tau", nv, 1)

# Compute kinematics casadi graphs
cpin.aba(cmodel, cdata, cq, cv, ctauq)
cpin.forwardKinematics(cmodel, cdata, cq, cv, caq)
cpin.updateFramePlacements(cmodel, cdata)

caba = casadi.Function("aba", [cx, ctauq], [cdata.ddq])


In [6]:
target_acc = np.array([0.0]*model.nv)

# q, dq, can be modified for different initial value
# x0 = np.array([0.0] * nx)
x0 = [9.63267947e-05,9.99999995e-01,  2.618, -1.57, -9.99999996e-01,  9.26535897e-05,  2.618,  1.0, 0.0,
      0.0,0.0,0.0,0.0,0.0,0.0]

error_fcn_kinova = casadi.Function(
    "error_fcn_kinova",
    [ctauq],
    [casadi.sumsqr(target_acc - caba(x0,ctauq))],
)

opti = casadi.Opti()
var_torque = opti.variable(nv,1)

opti.minimize(error_fcn_kinova(var_torque))
opti.subject_to(opti.bounded(-150, var_torque, 150))
opti.set_initial(var_torque, 100)
 
opti.solver('ipopt')
 
sol = opti.solve()
 
torque_result = sol.value(var_torque)
print(torque_result)
print('results acceleration')
print(error_fcn_panda(torque_result))
# print(error_fcn_panda([-3.49740277,-4.03988651e+00, -3.42179339e-09, -3.26685612e+00, -2.79824263e-09,  2.29967156e+00,  1.92950351e-11,  3.27483146e-10,-3.27483146e-10]))

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.4870793e+10 0.00e+00 5.00e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## UR3


In [7]:
robot = robex.load("ur3")

# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()

In [8]:
cmodel = cpin.Model(model)
cdata = cmodel.createData()

nq = model.nq
nv = model.nv
nx = nq + nv
ndx = nx


cx = casadi.SX.sym("x", nx, 1)
cdx = casadi.SX.sym("dx", nx, 1)
cq = cx[:nq]
cv = cx[nq:]
caq = casadi.SX.sym("a", nv, 1)
ctauq = casadi.SX.sym("tau", nv, 1)

# Compute kinematics casadi graphs
cpin.aba(cmodel, cdata, cq, cv, ctauq)
cpin.forwardKinematics(cmodel, cdata, cq, cv, caq)
cpin.updateFramePlacements(cmodel, cdata)

caba = casadi.Function("aba", [cx, ctauq], [cdata.ddq])


In [9]:
target_acc = np.array([0.0]*model.nv)

# q, dq, can be modified for different initial value
# x0 = np.array([0.0] * nx)
x0 = [0.0] * nx

error_fcn = casadi.Function(
    "error_fcn",
    [ctauq],
    [casadi.sumsqr(target_acc - caba(x0,ctauq))],
)

opti = casadi.Opti()
var_torque = opti.variable(nv,1)

opti.minimize(error_fcn(var_torque))
opti.subject_to(opti.bounded(-150, var_torque, 150))
opti.set_initial(var_torque, 100)
 
opti.solver('ipopt')
 
sol = opti.solve()
 
torque_result = sol.value(var_torque)
print(torque_result)
print('results acceleration')
print(error_fcn(torque_result))

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.4258477e+07 0.00e+00 5.00e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 